In [2]:
## Imports
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

In [4]:
test_set_features_df = pd.read_csv('test_set_features.csv')
train_set_features_df = pd.read_csv('training_set_features.csv')
train_set_labels_df = pd.read_csv('training_set_labels.csv')

### Read Data

In [17]:
## Training data
print(f"Train data {train_set_features_df.shape}")
print(f"Train columns {train_set_features_df.columns}")
print("")
print(f"Test data {test_set_features_df.shape}")

Train data (26707, 36)
Train columns Index(['respondent_id', 'h1n1_concern', 'h1n1_knowledge',
       'behavioral_antiviral_meds', 'behavioral_avoidance',
       'behavioral_face_mask', 'behavioral_wash_hands',
       'behavioral_large_gatherings', 'behavioral_outside_home',
       'behavioral_touch_face', 'doctor_recc_h1n1', 'doctor_recc_seasonal',
       'chronic_med_condition', 'child_under_6_months', 'health_worker',
       'health_insurance', 'opinion_h1n1_vacc_effective', 'opinion_h1n1_risk',
       'opinion_h1n1_sick_from_vacc', 'opinion_seas_vacc_effective',
       'opinion_seas_risk', 'opinion_seas_sick_from_vacc', 'age_group',
       'education', 'race', 'sex', 'income_poverty', 'marital_status',
       'rent_or_own', 'employment_status', 'hhs_geo_region', 'census_msa',
       'household_adults', 'household_children', 'employment_industry',
       'employment_occupation'],
      dtype='object')

Test data (26708, 36)


In [18]:
train_set_features_df.head()

,respondent_id,h1n1_concern,h1n1_knowledge,behavioral_antiviral_meds,behavioral_avoidance,behavioral_face_mask,behavioral_wash_hands,behavioral_large_gatherings,behavioral_outside_home,behavioral_touch_face,...,income_poverty,marital_status,rent_or_own,employment_status,hhs_geo_region,census_msa,household_adults,household_children,employment_industry,employment_occupation
0,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,...,Below Poverty,Not Married,Own,Not in Labor Force,oxchjgsf,Non-MSA,0.0,0.0,NaN,NaN
1,1,3.0,2.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,...,Below Poverty,Not Married,Rent,Employed,bhuqouqj,"MSA, Not Principle City",0.0,0.0,pxcmvdjn,xgwztkwe
2,2,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,"<= $75,000, Above Poverty",Not Married,Own,Employed,qufhixun,"MSA, Not Principle City",2.0,0.0,rucpziij,xtkaffoo
3,3,1.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,...,Below Poverty,Not Married,Rent,Not in Labor Force,lrircsnp,"MSA, Principle City",0.0,0.0,NaN,NaN
4,4,2.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,...,"<= $75,000, Above Poverty",Married,Own,Employed,qufhixun,"MSA, Not Principle City",1.0,0.0,wxleyezf,emcorrxb


In [20]:
# Check for NaN values
train_set_features_df.isnull().any()

respondent_id                  False
h1n1_concern                    True
h1n1_knowledge                  True
behavioral_antiviral_meds       True
behavioral_avoidance            True
behavioral_face_mask            True
behavioral_wash_hands           True
behavioral_large_gatherings     True
behavioral_outside_home         True
behavioral_touch_face           True
doctor_recc_h1n1                True
doctor_recc_seasonal            True
chronic_med_condition           True
child_under_6_months            True
health_worker                   True
health_insurance                True
opinion_h1n1_vacc_effective     True
opinion_h1n1_risk               True
opinion_h1n1_sick_from_vacc     True
opinion_seas_vacc_effective     True
opinion_seas_risk               True
opinion_seas_sick_from_vacc     True
age_group                      False
education                       True
race                           False
sex                            False
income_poverty                  True
m

Most of the columns have null values

Column Details:

<b>h1n1_concern</b> - Level of concern about h1n1 flu

<b>h1n1_knowledge</b> - Level of knowledge about h1n1 flu

<b>behavioral_antiviral_meds</b> - Has taken anitviral medications

<b>behavioral_avoidance</b> - Has avoided close contact with others with flu-like symptons

<b>behavioural_face_mask</b> - Has bought a face mask

<b>behavioral_wash_hands</b> - Has frequently washed hands or used sanitzer

<b>behavioral_large_gatherings</b> - Has reduced time at large gatherings

<b>behavioral_outside_home</b> - Has reduced contact with people outisde

<b>behavioral_touch_face</b> - Has avoided touching eyes, nose or mouth

<b>doctor_recc_h1n1</b> - h1n1 flu vaccine recommened by doctor

<b>doctor_recc_seasonal</b> - seasonal flu  vaccine recommeded by doctor

<b>chronic_med_condition</b> - Has chronic condition 

<b>child_under_6_months</b> - Has regular contact with child nder age 6

<b>health_worker</b> - Is a health worker

<b>health_insurance</b> - Has health insurance

<b>opinion_h1n1_vacc_effective</b> - Respondents opinion about vaccines effectives (1=Not at all effective, 2=Not very effective, 3=Dont know, 4=somewhat high, 5=very high

<b>opinion_seas_vacc_effective</b> - Respondents opinion about seasonal flu

<b>opinion_seas_risk</b> - Respondent's opinion about risk of getting sick with seasonal flu without vaccine

<b>opinion_seas_sick_from_vacc</b> - Respondent's worry of getting sick from taking seasonal flu vaccine

<b>age_group</b> - Age group of respondent

<b>education</b> - Self reported education level

<b>race</b> - race of respondent

<b>sex</b> - Sex of respodent

<b>income_poverty</b> - Household annual income

<b>martial_status</b> - Martial status of respodent

<b>hhs_geo_region</b> - Respondents residence

<b>census_msa</b> - Respondent's residence within metropolitan statistical areas (MSA) as defined by the U.S. Census

<b>household_adults</b> - Number of other adults in household

<b>household_children</b> - Number of children

<b>employment_industry</b> - Type of industry respondent is employed (string)

<b>employment_occupation</b> - Type of occupation of respodent (string)

In [21]:
train_set_features_df.head()

,respondent_id,h1n1_concern,h1n1_knowledge,behavioral_antiviral_meds,behavioral_avoidance,behavioral_face_mask,behavioral_wash_hands,behavioral_large_gatherings,behavioral_outside_home,behavioral_touch_face,...,income_poverty,marital_status,rent_or_own,employment_status,hhs_geo_region,census_msa,household_adults,household_children,employment_industry,employment_occupation
0,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,...,Below Poverty,Not Married,Own,Not in Labor Force,oxchjgsf,Non-MSA,0.0,0.0,NaN,NaN
1,1,3.0,2.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,...,Below Poverty,Not Married,Rent,Employed,bhuqouqj,"MSA, Not Principle City",0.0,0.0,pxcmvdjn,xgwztkwe
2,2,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,"<= $75,000, Above Poverty",Not Married,Own,Employed,qufhixun,"MSA, Not Principle City",2.0,0.0,rucpziij,xtkaffoo
3,3,1.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,...,Below Poverty,Not Married,Rent,Not in Labor Force,lrircsnp,"MSA, Principle City",0.0,0.0,NaN,NaN
4,4,2.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,...,"<= $75,000, Above Poverty",Married,Own,Employed,qufhixun,"MSA, Not Principle City",1.0,0.0,wxleyezf,emcorrxb


In [32]:
train_set_features_df.h1n1_concern.value_counts()

2.0    10575
1.0     8153
3.0     4591
0.0     3296
Name: h1n1_concern, dtype: int64

Most of the people are somewhat concerned about h1n1

In [33]:
train_set_features_df.h1n1_knowledge.value_counts()

1.0    14598
2.0     9487
0.0     2506
Name: h1n1_knowledge, dtype: int64

Most of the people have little knowledge

In [34]:
train_set_features_df.behavioral_antiviral_meds.value_counts()

0.0    25335
1.0     1301
Name: behavioral_antiviral_meds, dtype: int64

Most of the people haven't taken any anti viral meds

In [36]:
train_set_features_df.behavioral_face_mask.value_counts()

0.0    24847
1.0     1841
Name: behavioral_face_mask, dtype: int64

Most of the people havent purchased any mask

In [37]:
train_set_features_df.behavioral_wash_hands.value_counts()

1.0    22015
0.0     4650
Name: behavioral_wash_hands, dtype: int64

Most people do wash or sanitize their hands

In [38]:
train_set_features_df.behavioral_large_gatherings.value_counts()

0.0    17073
1.0     9547
Name: behavioral_large_gatherings, dtype: int64

Majority people avoid large gatherings

In [39]:
train_set_features_df.behavioral_outside_home.value_counts()

0.0    17644
1.0     8981
Name: behavioral_outside_home, dtype: int64

Most people have not reduced contact outside their home

In [40]:
train_set_features_df.behavioral_touch_face.value_counts()

1.0    18001
0.0     8578
Name: behavioral_touch_face, dtype: int64

In [41]:
train_set_features_df.behavioral_touch_face.value_counts()

1.0    18001
0.0     8578
Name: behavioral_touch_face, dtype: int64

In [42]:
train_set_features_df.doctor_recc_h1n1.value_counts()

0.0    19139
1.0     5408
Name: doctor_recc_h1n1, dtype: int64

In [43]:
train_set_features_df.doctor_recc_seasonal.value_counts()

0.0    16453
1.0     8094
Name: doctor_recc_seasonal, dtype: int64

In [44]:
train_set_features_df.child_under_6_months .value_counts()

0.0    23749
1.0     2138
Name: child_under_6_months, dtype: int64

In [46]:
train_set_features_df.opinion_h1n1_vacc_effective.value_counts()

4.0    11683
5.0     7166
3.0     4723
2.0     1858
1.0      886
Name: opinion_h1n1_vacc_effective, dtype: int64

In [ ]:
train_set_features_df.health_worker.value_counts()

In [45]:
train_set_features_df.health_worker.value_counts()

0.0    23004
1.0     2899
Name: health_worker, dtype: int64